# Обучение модели для сегментации мозга

## Подготовка среды

In [ ]:
import datetime
import os


%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation

plt.rcParams["animation.html"] = "jshtml"
import tensorflow as tf
from loader_data import PreprocessLoadData, AnimateView
from models.model_MultiReaUNet2d import Model2DMultiResUnet
from models.model_UNet2d import Model2DUnet
import numpy as np

num_gpu_device = len(tf.config.experimental.list_physical_devices('GPU'))
if num_gpu_device == 1:
    tf.config.set_visible_devices(tf.config.experimental.list_physical_devices('GPU')[0:1], 'GPU')
    gpu_device = tf.config.experimental.list_physical_devices('GPU')[0]
    tf.config.experimental.set_memory_growth(gpu_device, True)
else:
#     mirrored_strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.ReductionToOneDevice())
    mirrored_strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
    

    print('Количество GPU: {}'.format(mirrored_strategy.num_replicas_in_sync))




## Определяем ключевые параметры

 - **KERNEL_DIM** - Размер изображения которое подаем на вход сети. Независимо от размера исходного изображения, генератор будет делить исходное изображение на куски размером KERNEK_DIM
 - **STEP_WINDOW** - Разбиение исходного изображения на куски размером KERNEK_DIM происходит по принципу плавающего окна. STEP_WINDOW задает шаг смещения окна.
 - **BATCH_SIZE** - Количество кусков в одном батче
 - **Model** - Класс модели, которую будем обучать.

In [ ]:
STEP_WINDOW = (64, 64)
BATCH_SIZE = 16
KERNEL_DIM = (256, 256)

# Model2DMultiResUnet or Model2DUnet
Model = Model2DMultiResUnet


## Проверяем наличие данных для обучения

### Находим обучающие данные 

In [ ]:

loader = PreprocessLoadData(kernel_size=KERNEL_DIM, step=STEP_WINDOW, batch_size=BATCH_SIZE)
loader.find_files()
print(f"Был найден набор из {loader.length_data()} пар данных")


### Получаем генератор данных

In [ ]:
train_dataset = loader.get_generator_data("train",
                                          threshold=0.1,
                                          random_change_plane=False,
                                          augmentation=False,
                                          is_whitening=False
                                         )
test_dataset = loader.get_generator_data("test",
                                         threshold=0.1,
                                         random_change_plane=False,
                                         augmentation=False,
                                         is_whitening=False)


#### Проверяем как работает генератор

In [ ]:
def display(display_list):
    fig = plt.figure(figsize=(15, 5))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    list_ax = [fig.add_subplot(1, len(display_list), i + 1) for i in range(len(display_list))]
    list_exponenta = []
    for i in range(len(display_list)):
        plt.title(title[i])
        list_exponenta.append((display_list[i].min(),display_list[i].max()))
    list_ims = []
    for ind_z in range(display_list[0].shape[0]):
        if display_list[0][ind_z,...].max() == 0:
            continue
        ims = []
        for i in range(len(display_list)):
            cmap_val = 'gray' if i == 0 else "viridis"
            im = list_ax[i].imshow(tf.keras.preprocessing.image.array_to_img(display_list[i][ind_z,...]),
                           animated=True, cmap=cmap_val, vmin=list_exponenta[i][0], vmax=list_exponenta[i][1])
            ims.append(im)
        list_ims.append(ims)
    ani = animation.ArtistAnimation(fig,
                                    list_ims,
                                    interval=50,
                                    blit=True)
    return ani


for image, mask in train_dataset:
    print(image.shape)
    print(mask.shape)
    sample_image, sample_mask = image, mask
    break
display([sample_image, sample_mask])


## Создаем модель

 - n_channels: Размер канала входных данных.
 - initial_learning_rate: Начальная скорость обучения модели.
 - n_classes: Количество классов, которые изучает модель .
 - start_val_filters: Количество фильтров, которые будет иметь первый слой в сети.
 - list_metrics: Список метрик, для обучения модели.
 - type_up_convolution: тип повышающего слоя. up_sampling_3d использует меньше памяти.
 - pool_size: Размер пула для максимальных операций объединения. Целое число или кортеж.
 - input_img_shape: Форма входных данных кортеж(3N) или целое число (если форма соответствует кубу).
 - depth: глубина модели. Уменьшение глубины может уменьшить объем памяти, необходимый для тренировки.


In [ ]:
if num_gpu_device == 1:
    manager_model = Model(depth=5, start_val_filters=16, input_img_shape=KERNEL_DIM, n_classes=3, n_channels=3,
                                        initial_learning_rate=1.0)

else:
    with mirrored_strategy.scope():
        manager_model = Model(depth=5, start_val_filters=16, input_img_shape=KERNEL_DIM, n_classes=3, n_channels=3,
                                            initial_learning_rate=1.0)
model = manager_model.model




### Проверяем как работает не обученная модель

In [ ]:

for image, mask in train_dataset:
    print(image.shape)
    print(mask.shape)
    pred_mask = manager_model.model.predict(image)
    break
display([image, mask, pred_mask])


### Обучаем

In [ ]:

EPOCHS = 150

total_pieces = (192 - BATCH_SIZE) // BATCH_SIZE + 1
total_pieces *= (256 - KERNEL_DIM[-2]) // STEP_WINDOW[-2] + 1
total_pieces *= (256 - KERNEL_DIM[-1]) // STEP_WINDOW[-1] + 1

total_mri_study = (loader.length_data() * 0.8)
total_mri_study = 2

STEPS_PER_EPOCH = round(total_pieces * total_mri_study)
# STEPS_PER_EPOCH = round(STEPS_PER_EPOCH * 0.6)
VALIDATION_STEPS = round(2)

early_stop_val = 20

checkpoint_path = 'checkpoints2D//cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_directory = os.path.join("Results", timestamp)

# Добавление отображения через TensorBoard.
log_dir = os.path.join("Results",
                       timestamp,
                       "logs",
                       "fit",
                       datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
full_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                       save_weights_only=True,
                                       save_freq=STEPS_PER_EPOCH,
                                       verbose=1),
    tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                   histogram_freq=1,
                                   profile_batch=0,
                                   write_images=True),
    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=early_stop_val)]

model_history = manager_model.model.fit(train_dataset, epochs=EPOCHS,
                                        steps_per_epoch=STEPS_PER_EPOCH,
                                        validation_steps=VALIDATION_STEPS,
                                        validation_data=test_dataset,
                                        verbose=1,
                                        callbacks=full_callbacks)

manager_model.model.save(f"train.{Model.__name__}_{KERNEL_DIM}.{timestamp}")


In [ ]:
show_predictions(dataset=train_dataset, num=3)